# **YouTube Search**
pip install youtube-search  
pip install youtube-transcript-api

## YoutubeSearch

In [1]:
from youtube_search import YoutubeSearch

query = "인공지능 에이전트"
# query = "What is RAG?"
videos = YoutubeSearch(query, max_results=3)
videos

In [2]:
videos = videos.to_dict()
videos

[{'id': '1s35ZBwXn34',
  'thumbnails': ['https://i.ytimg.com/vi/1s35ZBwXn34/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLBI8n1HOYbUYGroq_XkclvoPU-MOA',
   'https://i.ytimg.com/vi/1s35ZBwXn34/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLANvMXIXs4rurZAq6H5X7jF1UyPlg'],
  'title': '인간 직업 70% 사라진다, AI에이전트가 바꿀 미래 시나리오  / 구글 제미나이 충격!  환각’ 줄인 AI, 1년마다 100배 똑똑해진다ㅣ솔트룩스 이경일 대표 (풀버전)',
  'long_desc': None,
  'channel': '815머니톡',
  'duration': '55:33',
  'views': '조회수 45,539회',
  'publish_time': '3개월 전',
  'url_suffix': '/watch?v=1s35ZBwXn34&pp=ygUZ7J246rO17KeA64qlIOyXkOydtOyghO2KuA%3D%3D'},
 {'id': 'u1V7wJMX0r8',
  'thumbnails': ['https://i.ytimg.com/vi/u1V7wJMX0r8/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLA_SfLgqq7xIrRbWigy7oEvxzGzhA',
   'https://i.ytimg.com/vi/u1V7wJMX0r8/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLD4O70w1Op_f--XML3HlXRxBV7R6Q'],
  'title': '에이 설마 내 밥줄이 끊길까... 심상치 않은

In [ ]:
len(videos)

In [ ]:
# 너무 긴 영상 걸러내기: 60분 이하
print('총 영상 수:', len(videos))

filtered_videos = []
for v in videos:
    if type(v['duration']) == str:
        if len(v['duration'].split(':')) < 3:
            filtered_videos.append(v)

print('60분 이하 영상 수:', len(filtered_videos))

In [ ]:
# 비디오 id 추출
video_ids = [v['id'] for v in filtered_videos]
video_ids

## YouTubeTranscriptApi

### 검색된 영상의 자막 가져오기

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

ytt = YouTubeTranscriptApi()

fetched_transcript = ytt.fetch(video_ids[0], languages=['ko', 'en'])

In [ ]:
fetched_transcript

In [ ]:
full_transcript = ""
for segment in fetched_transcript:
    full_transcript += segment.text + " "
print(f"총글자수: {len(full_transcript)}")
print(full_transcript)

### 자막 번역하기

In [ ]:
transcript_list = ytt.list(video_ids[0])
transcript_list

In [ ]:
transcript = transcript_list.find_transcript(['en'])
transcript

In [ ]:
print(
    transcript.video_id,
    transcript.language,
    transcript.language_code, 
    transcript.is_generated, # whether it has been manually created or generated by YouTube
    transcript.is_translatable, # whether this transcript can be translated or not
    transcript.translation_languages, # a list of languages the transcript can be translated to
)

In [ ]:
fetched_transcript = transcript.fetch()

full_transcript = ""
for segment in fetched_transcript:
    full_transcript += segment.text + " "

full_transcript

In [ ]:
# 번역하기
translated_transcript = transcript.translate('ko')

In [ ]:
fetched_transcript = translated_transcript.fetch()

full_ko_transcript = ""
for segment in fetched_transcript:
    full_ko_transcript += segment.text + " "

full_ko_transcript

## 유튜브 검색하고 답하는 에이전트

In [ ]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from youtube_search import YoutubeSearch
from youtube_transcript_api import YouTubeTranscriptApi
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

# llm 모델 선정
llm = ChatOpenAI(model="gpt-4o-mini")

# 유튜브 검색
query = "What is LangGraph?"

videos = YoutubeSearch(query, max_results=3).to_dict()

# 너무 긴 영상 걸러내기: 60분 이하
print('총 영상 수:', len(videos))

filtered_videos = []
for v in videos:
    if type(v['duration']) == str:
        if len(v['duration'].split(':')) < 3:
            filtered_videos.append(v)

print('60분 이하 영상 수:', len(filtered_videos))

# 비디오 id 추출
video_ids = [v['id'] for v in filtered_videos]

In [ ]:
# 자막 추출
ytt = YouTubeTranscriptApi()
fetched_transcript = ytt.fetch(video_ids[1], languages=['ko', 'en'])

full_transcript = ""
for segment in fetched_transcript:
    full_transcript += segment.text + " "

In [ ]:
# 프롬프트 템플릿 작성
prompt = ChatPromptTemplate.from_template(
    """
    당신은 사용자의 질문에 친절히 답하는 조력자입니다. 주어진 유튜브 검색 영상의 자막을 이용해 한국어로 질문에 답하세요.
    핵심만 간결하게 정리해서 markdown 형식으로 답하고 출처도 표시할 수 있으면 해주세요.

    질문: {query}

    검색결과: {contents}
    """
)
# 체인으로 연결하여 에이전트 구성
agent = prompt | llm | StrOutputParser()

# 유튜브 검색 기반 답변 구하기
response = agent.invoke({"query": query, "contents": full_transcript})

print(response)

## LangGraph 개요

- **정의**: LangGraph는 Lang chain에 의해 개발된 AI 에이전트 구축 프레임워크입니다. 고도로 유연하여 언어 모델을 연결하고 다음에 어떤 작업이 수행될지를 제어할 수 있습니다.
  
- **프로그래밍 언어**: Python 및 JavaScript에서 지원되며, 2023년 10월에 출시되었습니다.

- **주요 특징**:
  - **스트리밍 지원**: 메시지 또는 토큰 기반으로 실시간 처리 가능.
  - **비동기 실행**: 여러 작업을 동시에 수행할 수 있음.
  - **상태 지속성**: 데이터베이스를 사용해 상태를 유지할 수 있음.
  - **인간 개입 실행**: 중요한 작업에 대해 사람의 승인을 요구할 수 있음.
  
- **응용 사례**:
  - 고객 지원 에이전트
  - 정보 수집 및 보고서 작성
  - 맞춤형 분석 도구
  - 개인화된 추천 시스템
  - 최신 트렌드 또는 자료 연구

- **구성 요소**:
  - **노드**: 실행할 코드를 포함.
  - **에지**: 어떤 노드가 다음에 실행될지를 결정.
  - **상태**: 입력, 출력 및 변수를 저장.

출처: 유튜브 자막 요약

In [ ]:
# 모든 비디오에 대해 요약 답변 생성
from tqdm import tqdm # tqdm은 진행 상황을 보여주는 라이브러리

for v in tqdm(filtered_videos):
    fetched_transcript = ytt.fetch(v['id'], languages=['ko', 'en'])
    full_transcript = ""
    for segment in fetched_transcript:
        full_transcript += segment.text + " "    

    v['summary'] = agent.invoke({"query": query, "contents": full_transcript})
   
filtered_videos